In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import numpy as np
import chess
import chess.engine
import chess.pgn
import io
import zstandard as zstd
import re

from bitboard import bitboard_to_array, bitboards_to_3d_array, board_to_bitboards, array_to_bitboard, bitboards_to_board


# input shape for the board is 12x8x8
board_input = layers.Input(shape=(12, 8, 8))

# input for the move number is a single scalar
move_number_input = layers.Input(shape=(1,))

# Add several convolutional layers to process the board
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(board_input)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Flatten()(x)
# Concatenate the processed board and move number inputs
x = layers.Concatenate()([x, move_number_input])  # move_number_input is directly used here

# Add more fully-connected layers
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
# Add an output layer, assuming we're doing regression to predict the evaluation score
output_layer = layers.Dense(1)(x)

# Create the model
model = tf.keras.Model(inputs=(board_input, move_number_input), outputs=output_layer)

# Then, you can initialize it:
dummy_input_1 = tf.random.uniform([1, 12, 8, 8])
dummy_input_2 = tf.random.uniform([1])
_ = model([dummy_input_1, dummy_input_2])

model.load_weights('./data/my_model-100-100-32.h5')

def evaluate_board(board):
    piece_values = {chess.PAWN: 10, chess.KNIGHT: 30, chess.BISHOP: 30, chess.ROOK: 50, chess.QUEEN: 90, chess.KING: 900}
    center_squares = [chess.D4, chess.E4, chess.D5, chess.E5]
    center_control = [0, 0]  # [white's control, black's control]

    pawn_ranks = [1, 6]  # rank-2 for white, rank-7 for black
    knight_files = ['b', 'g']  # knight's starting files
    bishop_files = ['c', 'f']  # bishop's starting files
    development = [0, 0]  # [white's development, black's development]

    mobility = [0, 0]  # [white's mobility, black's mobility]

    pawn_structure = [0, 0]  # [white's pawn structure, black's pawn structure]
    king_safety = [0, 0]  # [white's king safety, black's king safety]

    score = 0

    for piece_type in piece_values:
        for color in [chess.WHITE, chess.BLACK]:
            pieces = board.pieces(piece_type, color)
            piece_value = piece_values[piece_type] * (-1 if color == chess.BLACK else 1)
            for piece in pieces:
                score += piece_value
                if piece in center_squares:
                    center_control[color] += piece_value
                if piece_type == chess.PAWN and chess.square_rank(piece) in pawn_ranks:
                    development[color] += piece_value / 2
                if piece_type == chess.KNIGHT and chess.square_file(piece) in knight_files:
                    development[color] += piece_value / 2
                if piece_type == chess.BISHOP and chess.square_file(piece) in bishop_files:
                    development[color] += piece_value / 2

    # Assess pawn structures
    for color in [chess.WHITE, chess.BLACK]:
        pawns = board.pieces(chess.PAWN, color)
        if len(pawns) > 0:
            average_square = sum([chess.square_rank(p) for p in pawns]) / len(pawns)
            pawn_structure[color] = len(pawns) / (1 + average_square)

    # Assess king safety
    for color in [chess.WHITE, chess.BLACK]:
        king = board.king(color)
        if king:
            king_safety[color] = len(board.attackers(not color, king))

    # Assess mobility
    for color in [chess.WHITE, chess.BLACK]:
        copy_board = board.copy()
        copy_board.turn = color
        mobility[color] = len(list(copy_board.legal_moves))

    # Combine evaluations
    score += center_control[chess.WHITE] * 0.1  # weigh white's center control
    score -= center_control[chess.BLACK] * 0.1  # weigh black's center control (negative because it's bad for white)

    score += development[chess.WHITE] * 0.05  # weigh white's piece development
    score -= development[chess.BLACK] * 0.05  # weigh black's piece development (negative because it's bad for white)

    score += mobility[chess.WHITE] * 0.03  # weigh white's mobility
    score -= mobility[chess.BLACK] * 0.03  # weigh black's mobility (negative because it's bad for white)

    score += pawn_structure[chess.WHITE] * 0.02  # weigh white's pawn structure
    score -= pawn_structure[chess.BLACK] * 0.02  # weigh black's pawn structure (negative because it's bad for white)

    score -= king_safety[chess.WHITE] * 0.5  # weigh white's king safety (negative because unsafe king is bad)
    score += king_safety[chess.BLACK] * 0.5  # weigh black's king safety (positive because unsafe king is good for white)

    return score



number_of_games = 0

stop = 100

engine = chess.engine.SimpleEngine.popen_uci("/home/jerry/Stockfish/src/stockfish")
with open('./data/lichess_db_standard_rated_2023-06.pgn.zst', 'rb') as f:
    dctx = zstd.ZstdDecompressor()
    reader = dctx.stream_reader(f)
    text_stream = io.TextIOWrapper(reader, encoding='utf-8')

    # Iterate over all games in the PGN file
    while True:
        dataitems = []
        game = chess.pgn.read_game(text_stream)
        if game is None:
            break

        # Check if the game has embedded Stockfish evaluations
        has_evaluations = any('[%eval ' in node.comment for node in game.mainline())
        if not has_evaluations:
            print("found game with no eval")
            continue
            
        number_of_games += 1
        
        # Setup initial empty chess board
        board = chess.Board()
        
        # First color to play is white (True in the COLORS list)
        to_play_next = True
        move_number = 0
        # Iterate through all moves of the game
        for node in game.mainline():
            move_number += 1
            move = node.move
            board.push(move)
            print(f"game: {number_of_games} move: {move_number} {move.uci()}")
            print(board)
            
            # Get the comment (which contains the evaluation) for this move
            comment = node.comment
            match = re.search(r"\[%eval (.*?)\]", comment)
            if match is None:
                print(f"did not find a match when one was expected.")
                continue
            
            score_str = match.group(1)
            
            # Convert mate in 'n' moves to large scores
            if '#' in score_str:
                if '-' in score_str:
                    score = -1000
                else:
                    score = 1000
            else:
                score = float(score_str)

            move_number_int = np.int16(board.ply())
            positions = []
            pp = []
            bitboards = board_to_bitboards(board)
            bb = bitboards_to_3d_array(bitboards)
            positions.append(bb)
            pp.append(move_number_int)
            bb = np.array(positions)
            pp = np.array(pp)
            prediction = model.predict((bb, pp), verbose = 0)

            # Convert the board position to bitboards and add to list
            bitboards = board_to_bitboards(board)
            dataitems.append((((bitboards_to_3d_array(bitboards), np.int16(board.ply()))), np.float16(score)))

            # Evaluate the position using Stockfish
            info = engine.analyse(board, chess.engine.Limit(time=10.0))
            # The 'score' field in the info dictionary contains the evaluation of the position.
            # We need to convert it to a float, and handle the case where the score is
            # reported as a mate in 'n' moves.
            relative_score = info["score"].relative.score()
            if info["score"].is_mate():
                if relative_score is not None and relative_score > 0:
                    stock_score = float('inf')
                else:
                    stock_score = float('-inf')
            else:
                if relative_score is not None:
                    stock_score = relative_score / 100.0  # convert from centipawns to full pawn units
                else:
                    stock_score = 0.0  # or any other default value you find appropriate
            
            print(f"score: {score} predicted: {prediction[0][0]} evaluation: {evaluate_board(board)} stockfish: {stock_score}")
            print("----------------------------------------")

        if number_of_games == stop:
            break


                        
                    


found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
found game with no eval
game: 1 move: 1 e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
score: 0.36 predicted: 2.3933348655700684 evaluation: 5.098253968253968 stockfish: -0.38
----------------------------------------
game: 1 move: 2 c7c5
r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
score: 0.32 predicted: 1.6423670053482056 evaluation: 4.787407407407

In [ ]:
import tensorflow as tf
import numpy as np
import io
import chess.pgn
import zstandard as zstd
import re

from bitboard import bitboard_to_array, bitboards_to_3d_array, board_to_bitboards, array_to_bitboard, bitboards_to_board


print_frequency = 10

def data_generator():
    global number_of_games
    number_of_games = 0
    global print_frequency
    
    with open('./data/lichess_db_standard_rated_2023-06.pgn.zst', 'rb') as f:
        dctx = zstd.ZstdDecompressor()
        reader = dctx.stream_reader(f)
        text_stream = io.TextIOWrapper(reader, encoding='utf-8')

        # Iterate over all games in the PGN file
        while True:
            dataitems = []
            game = chess.pgn.read_game(text_stream)
            if game is None:
                break
    
            # Check if the game has embedded Stockfish evaluations
            has_evaluations = any('[%eval ' in node.comment for node in game.mainline())
            if not has_evaluations:
                print("found game with no eval")
                continue
                
            number_of_games += 1
            
            # Setup initial empty chess board
            board = chess.Board()
            
            # First color to play is white (True in the COLORS list)
            to_play_next = True
            move_number = 0
            # Iterate through all moves of the game
            for node in game.mainline():
                move_number += 1
                move = node.move
                board.push(move)
                print(f"game: {number_of_games} move: {move_number} {move.uci()}")
                print(board)
                
                # Get the comment (which contains the evaluation) for this move
                comment = node.comment
                match = re.search(r"\[%eval (.*?)\]", comment)
                if match is None:
                    print(f"did not find a match when one was expected.")
                    continue
                
                score_str = match.group(1)
                
                # Convert mate in 'n' moves to large scores
                if '#' in score_str:
                    if '-' in score_str:
                        score = -1000
                    else:
                        score = 1000
                else:
                    score = float(score_str)
                
                # Convert the board position to bitboards and add to list
                bitboards = board_to_bitboards(board)
                dataitems.append((((bitboards_to_3d_array(bitboards), np.int16(to_play_next))), np.float32(score)))
                to_play_next = not to_play_next
                if number_of_games % print_frequency == 0:
                    print(f"Generated {number_of_games} games so far")
                    
            for item in dataitems:
                yield item 

i = 0
for item in data_generator():
    i += 1
    if i == 200:
        break